In [2]:
import cv2
import time
import threading
import queue
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import speech_recognition as sr
from deepface import DeepFace

# Function to capture video from the camera
def capture_video(result_queue):
    cap = cv2.VideoCapture(0)  # The zero signifies the default camera.
    capture_duration = 10  # in seconds
    end_time = time.time() + capture_duration
    emotion_score=[]
    cv2.namedWindow('Video', cv2.WINDOW_NORMAL)

    while time.time() < end_time:
        ret, frame = cap.read()  # To capture a frame
        if not ret:
            break

        cv2.imshow('Video', frame)
        
        # Allow the window to process events for a short period (e.g., 1 millisecond)
        cv2.waitKey(1)

        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        result = DeepFace.analyze(frame, actions=['emotion'],enforce_detection=False )
        emotion = result[0]['emotion']
        emotion.pop('neutral', None)
        emotion_score.append(emotion)
        #print(f'Emotion: {emotion}')

        


    cap.release()
    cv2.destroyAllWindows()
    total_scores = {emotion: 0.0 for emotion in emotion_score[0]}

    # Iterate through the emotion dictionaries and accumulate the scores
    for scores in emotion_score:
        for emotion, score in scores.items():
            total_scores[emotion] += score

    # Calculate the average by dividing each accumulated score by the number of records
    num_records = len(emotion_score)
    average_scores = {emotion: total_score / num_records for emotion, total_score in total_scores.items()}

    print("Average Emotion Scores:", average_scores)
    result_queue.put((average_scores))
    global max_value
    max_value = max(average_scores, key=average_scores.get)
    
    print("Your emotion is: ",max_value)
    return max_value


# Function to capture audio and perform sentiment analysis
def capture_audio(q):
    recognizer = sr.Recognizer() 

    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source, duration=1)
        print('Waiting for your message')
        recorded_audio = recognizer.listen(source,timeout=15)
        print('Done recording')
        
    text = ""  # Initialize text

    try:
        print('Printing the message...')
        text = recognizer.recognize_google(recorded_audio, language='en-US')
        print('Your message: {}'.format(text))
        q.put(text)
    except Exception as ex:
        print(ex)
    
    # Only perform sentiment analysis if text is not empty
    if text:
        Sentence = [str(text)]
        analyzer = SentimentIntensityAnalyzer()

        for i in Sentence:  
            v = analyzer.polarity_scores(i) 
            v.pop('compound',None)
            print(v)
        global max_value2
        max_value2 = max(v, key=v.get)
        print("Your tone is: ",max_value2)
        return v,max_value2
            

if __name__ == "__main__":
    q = queue.Queue()
    result_queue = queue.Queue()

    # Create threads for video and audio capture
    video_thread = threading.Thread(target=capture_video, args=(result_queue,))
    audio_thread = threading.Thread(target=capture_audio, args=(q,))
    
    # Start both threads
    video_thread.start()
    audio_thread.start()
    
    # Wait for both threads to finish
    video_thread.join()
    audio_thread.join()
    video_results = result_queue.get()
    average_score=video_results
   # print("It works", average_score)
    print("Audio",max_value2)
    print("Video",max_value)
    fin1 = max_value2 + max_value
    final = {'neuangry':'Try to calm down',
    'neudisgust':'Slight discomfort',
    'neufear':'frightened by something',
    'neuhappy':'Seems happy',
    'neusad':'Feeling down. Cheer up!',
    'neusurprise':'surprised by something',
    'negangry':'You seem frustrated',
    'negdisgust':'Disgusted by something in the environment',
    'negfear':'Frightened by something and might be in danger.',
    'neghappy':'Inconclusive',
    'negsad':'Feeling blue. If depressed, you should get some help.',
    'negsurprise':'Might have heard bad news',
    'posangry':'Inconclusive',
    'posdisgust':'Inconclusive',
    'posfear':'Might be the adrenaline, or might be incinclusive.',
    'poshappy':'Seems happy. Keep having an amazing day!',
    'possad':'Might be powering through difficult times',
    'possurprise':'Heard some good news or was pleasantly surprised'
            }
    if fin1 in final.keys():
        print("\nOUTPUT")
        print("--------------------")
        print(final[fin1])
        print("--------------------")
        
    

    

Waiting for your message
Done recording
Printing the message...
Your message: I am feeling so lonely
{'neg': 0.419, 'neu': 0.388, 'pos': 0.194}
Your tone is:  neg


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 24.06it/s]


Average Emotion Scores: {'angry': 14.142142147463876, 'disgust': 0.21097836732263497, 'fear': 16.965082870662698, 'happy': 2.4256817164418663, 'sad': 60.70179010405645, 'surprise': 0.043736164017279006}
Your emotion is:  sad
Audio neg
Video sad

OUTPUT
--------------------
Feeling blue. If depressed, you should get some help.
--------------------


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [5]:
data = pd.read_csv(r'C:\Users\Lenovo\Desktop\College\HCI\emotion_list.csv')
data.head()

,Happy,Sad,Disgust,Fear,Surprise,Angry,Speech_Polarity,Flag
0,77.350968,0.218367,0.000161,0.005019,0.014861,0.052936,0.4404,happy
1,1.228430,1.130087,0.001031,0.046237,0.078255,0.514148,-0.3182,sad
2,0.009387,79.948650,2.452688,5.261097,0.002699,0.012124,-0.5256,sad
3,0.373128,0.955328,2.097232,2.046810,0.119421,1.203636,0.2732,fear
4,0.514633,10.620252,4.391215,12.737787,0.034690,5.562527,-0.6249,fear


In [6]:
X = data.drop('Flag', axis=1)  
y = data['Flag']  

In [7]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the dataset into a training set and a testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create a decision tree classifier
clf = DecisionTreeClassifier()

# Train the classifier on your training data
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Classification Report:\n", report)
print("Confusion Matrix:\n", cm)

Accuracy: 0.375
Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.67      0.67      0.67         3
           3       0.50      0.50      0.50         2
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1

    accuracy                           0.38         8
   macro avg       0.23      0.23      0.23         8
weighted avg       0.38      0.38      0.38         8

Confusion Matrix:
 [[0 0 0 1 0]
 [0 2 0 1 0]
 [0 0 1 1 0]
 [0 0 1 0 0]
 [0 1 0 0 0]]


In [8]:
y_pred

array([3, 4, 3, 4, 2, 2, 2, 4])